# Feature Engineering

For some definitions, handling outliers and missing values, scaling, and encoding may be considered feature engineering. Here we'll draw a distinction between data preparation, data preprocessing, and feature engineering.

- **data preparation**: the basic data cleaning necessary to get our data ready for exploration/analysis, e.g. correcting data types, fixing typos
- **data preprocessing**: further data transformation done for the sake of modeling, as oppsoed to exploration/analysis, e.g. scaling, imputing, encoding
- **feature engineering**: adding, combining, or removing features; usually with the help of domain knowledge

Feature engineering can happen as part of data exploration or modeling, and engineered featured are also commonly explored.

Some examples of feature engineering by this definition:

- domain-based conversion (example: farenheit to celsius, BMI calculation, log transformation)
- domain based cutoffs (example: age >= 18 = is_adult; also dates)

- add / subtract (example: zillow dataset: beds + baths = room_count; total_sqft - 200 * bedrooms - 40 * bathrooms = living_area)
- combine as booleans as a count (example: telco_churn: streaming + backups + ...  = service_count)
- multiply / divide (example: tips dataset: total_bill / size = price_per_person)
- ratios (example: tips dataset: tip / total_bill = tip percentage)

Simplify!

- categorical with many unique values to top 3 + "Other"
- categorical to boolean: pool count -> has pool
- continous -> categorical via binning (aka quantization or discretization) (example: income -> high, medium, low earner)

In this lesson we'll cover some *automated* **feature selection** methods, that is, methods for determining which features are the most important.

- SelectKBest
- Recursive Feature Elimination
- Sequential Feature Selection

In [7]:
mycols = ['col1', 'col2', 'col3', 'col4']

In [10]:
# # start with initializing empty lists
# # feature space will contain our features
# feature_space = []
# # scores will contain model performance
# scores = []
# # iterate through the list of columns:
# for col in mycols:
#     # add a new feature in each cycle
#     feature_space.append(col)
#     # fit a model based on the new feature space
#     model_object.fit(X[feature_space], y)
#     # get the score for that model
#     scores.append(model_object.score(X[feature_space], y))
    
# my_scores = dict(zip(feature_space, scores))

## Setup

In [12]:
import numpy as np
import pandas as pd
import wrangle
from sklearn.feature_selection import SelectKBest, RFE, f_regression, SequentialFeatureSelector

In [13]:
df = wrangle.wrangle_grades()

In [14]:
# train_validate, test = train_test_split(df)
# train, validate = train_test_split(train_validate)

train, validate, test = wrangle.split_continuous(df)

train -> (56, 5)
validate -> (25, 5)
test -> (21, 5)


In [15]:
train.head(3)

,student_id,exam1,exam2,exam3,final_grade
51,52,70,75,78,72
24,25,57,65,75,65
57,58,79,70,85,81


In [16]:
X_train, y_train = train[['exam1', 'exam2', 'exam3']], train.final_grade
X_validate, y_validate = validate[['exam1', 'exam2', 'exam3']], validate.final_grade
X_test, y_test = test[['exam1', 'exam2', 'exam3']], test.final_grade

## Select K Best

- looks at each feature in isolation against the target based on correlation
- fastest of all approaches covered in this lesson
- doesn't consider feature interactions
- After fitting: `.scores_`, `.pvalues_`, `.get_support()`, and `.transform`

In [18]:
# make the thing
kbest = SelectKBest(f_regression, k=2)
# fit the thing
_ = kbest.fit(X_train, y_train)

In [23]:
# statistical f-value:
kbest.scores_
#p value: 
kbest.pvalues_

array([1.00651044e-43, 2.70194813e-25, 6.39985139e-29])

In [24]:
kbest_results = pd.DataFrame(
    dict(p=kbest.pvalues_, f=kbest.scores_),
                             index = X_train.columns)

In [25]:
kbest_results

,p,f
exam1,1.006510e-43,1892.759696
exam2,2.701948e-25,351.182258
exam3,6.399851e-29,497.594063


In [28]:
# get-support() will output a boolean mask to tell me which features were selected
# we can apply this mask to the columns in our original dataframe
X_train.columns[kbest.get_support()]

Index(['exam1', 'exam3'], dtype='object')

In [30]:
# kbest transform will convert our information to the selected feature subspace
# ****buuuuuut, its just a numpy array
kbest.transform(X_train)[:5]

array([[70, 78],
       [57, 75],
       [79, 85],
       [98, 96],
       [83, 86]])

In [31]:
X_train_transformed = pd.DataFrame(
    kbest.transform(X_train),
    columns=X_train.columns[kbest.get_support()],
    index=X_train.index
)

In [32]:
X_train_transformed.head(3)

,exam1,exam3
51,70,78
24,57,75
57,79,85


## RFE

- Recursive Feature Elimination
- Progressively eliminate features based on importance to the model
- Requires a model with either a `.coef_` or `.feature_importances_` property
- After fitting: `.ranking_`, `.get_support()`, and `.transform()`

In [34]:
from sklearn.linear_model import LinearRegression

In [36]:
# make a model object to use in RFE process.
# The model is here to give us metrics on feature importance and model score
# allowing us to recursively reduce the number of features to reach our desired space
model = LinearRegression()

In [37]:
# make thing
rfe = RFE(model, n_features_to_select=2)
# fit thing
rfe.fit(X_train, y_train)

RFE(estimator=LinearRegression(), n_features_to_select=2)

In [38]:
rfe.ranking_

array([1, 2, 1])

In [39]:
pd.DataFrame(
{
    'rfe_ranking': rfe.ranking_
},index = X_train.columns)

,rfe_ranking
exam1,1
exam2,2
exam3,1


In [40]:
rfe.get_support()

array([ True, False,  True])

In [45]:
X_train_transformed = pd.DataFrame(
rfe.transform(X_train),
index = X_train.index,
columns = X_train.columns[rfe.support_])

In [46]:
X_train_transformed.head()

,exam1,exam3
51,70,78
24,57,75
57,79,85
92,98,96
81,83,86


## Sequential Feature Selector

- progressively adds features based on cross validated model performance
- forwards: start with 0, add the best additional feature until you have the desired number
- backwards: start with all features, remove the worst performing until you have the desired number
- After fitting: `.support_`, `.transform`

In [44]:
model = LinearRegression()
sfs = SequentialFeatureSelector(model, n_features_to_select=2)
sfs.fit(X_train, y_train)

SequentialFeatureSelector(estimator=LinearRegression(), n_features_to_select=2)

In [47]:
X_train_transformed = pd.DataFrame(
sfs.transform(X_train),
index = X_train.index,
columns = X_train.columns[sfs.support_])

In [49]:
X_train_transformed

,exam1,exam3
51,70,78
24,57,75
57,79,85
92,98,96
81,83,86
42,83,86
101,58,70
97,92,94
10,58,70
54,85,87


## Conclusion

- Simpler models handle change + variability better
- Use RFE to narrow down your features and find the best ones, if your dataset is large (> 1GB; `df.info()`) use select k best instead
- Remember: feature engineering is much more than feature selection!